In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
input_fname = 'test_IOTA.in'
np.set_printoptions(precision=3)

# Prepare Beam and Lattice

In [2]:
beam, lattice0 = impact.readInputFile(input_fname)
beam.nCore_y=1
beam.nCore_z=1
beam.n_particles=8
beam.current=0.0
beam.integrator = 'NonLinear'
#beam.integrator = 'Linear'

for i in range(len(lattice0)):
    if lattice0[i].type == 'nonlinear_insert':
        elem_NL = copy(lattice0[i])
        break
elem_NL.strength_t = 100.0
print(elem_NL)

reading ImpactZ input file (test_IOTA.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done

             length: 1.8 [m]
           n_sckick: 96 [1]
              n_map: 5 [1]
         strength_t: 100.0
 transverse_scale_c: 0.01
       tune_advance: 0.3034496449165134
        pipe_radius: 10.0 [m]
               type: 'nonlinear_insert'


### NL vs Drift

##### nonlinear insert

In [3]:
#beam.integrator = 'NonLinear'
dist = beam.distribution
dist.distribution_type = 'Waterbag'
lattice_NL = [impact.getElem('loop_through_lattice'),
              impact.getElem('write_raw_ptcl'),
              elem_NL,
              impact.getElem('write_raw_ptcl')]
lattice_NL[0].nturn = 1
lattice_NL[1].file_id = 1001
lattice_NL[3].file_id = 1002

ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

impact.writeInputFile(beam,lattice_NL);
impact.run()
pData_NL0 = impact.readParticleData(1001,ke,mass,freq)
pData_NL1 = impact.readParticleData(1002,ke,mass,freq)

input error <- sum(beam.multi_charge.n_particles) not qual to beam.n_particles
  ... enforcing  beam.multi_charge.n_particles[0] to beam.n_particles
input error <- beam.multi_charge.current[0] not qual to beam.current
  ... enforcing  beam.multi_charge.current[0] to beam.current


##### drift

In [4]:
lattice_drift = copy(lattice_NL)
lattice_drift[2] = impact.getElem('drift')
lattice_drift[2].length = lattice_NL[2].length
lattice_drift[2].n_sckick = elem_NL.n_sckick

impact.writeInputFile(beam,lattice_drift);
impact.run()
pData_drift0 = impact.readParticleData(1001,ke,mass,freq)
pData_drift1 = impact.readParticleData(1002,ke,mass,freq)

##### check

In [5]:
print(pData_drift1[:,:6]-pData_NL1[:,:6])

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


### quad vs Drift

##### quad

In [6]:
lattice_quad = copy(lattice_NL)
lattice_quad[2] = impact.getElem('quad')
lattice_quad[2].length = 0.05
lattice_quad[2].B1 = 10.0
lattice_quad[2].n_sckick = 10

impact.writeInputFile(beam,lattice_quad);
impact.run()
pData_quad0 = impact.readParticleData(1001,ke,mass,freq)
pData_quad1 = impact.readParticleData(1002,ke,mass,freq)

##### drift

In [7]:
lattice_drift = copy(lattice_quad)
lattice_drift[2] = impact.getElem('drift')
lattice_drift[2].length = lattice_quad[2].length
lattice_drift[2].n_sckick = lattice_quad[2].n_sckick


impact.writeInputFile(beam,lattice_drift);
impact.run()
pData_drift0 = impact.readParticleData(1001,ke,mass,freq)
pData_drift1 = impact.readParticleData(1002,ke,mass,freq)

##### check

In [8]:
print(pData_drift1[:,:6]-pData_quad1[:,:6])

[[-1.151e-04 -4.518e-03  9.430e-05  3.781e-03 -2.231e-04  0.000e+00]
 [ 1.171e-04  4.594e-03 -4.612e-05 -1.852e-03 -2.485e-04  0.000e+00]
 [-5.389e-05 -2.149e-03  5.592e-05  2.258e-03 -2.318e-05  0.000e+00]
 [ 6.955e-05  2.763e-03 -6.672e-05 -2.693e-03 -5.008e-05  0.000e+00]
 [-1.693e-04 -6.691e-03  3.545e-05  1.458e-03 -3.229e-04  0.000e+00]
 [-6.216e-05 -2.407e-03 -1.010e-05 -3.959e-04 -1.197e-04  0.000e+00]
 [-1.452e-04 -5.702e-03  3.652e-05  1.474e-03 -3.604e-04  0.000e+00]
 [ 1.955e-04  7.699e-03  1.302e-05  5.178e-04 -5.203e-04  0.000e+00]]


### Linear vs NonLinear Integrator on quad

In [9]:
beam.integrator = 'Linear'
impact.writeInputFile(beam,lattice_quad);
impact.writeInputFile(beam,lattice_quad,'testLinear.in');
impact.run()
pData_linear1 = impact.readParticleData(1002,ke,mass,freq)
beam.integrator = 'NonLinear'
impact.writeInputFile(beam,lattice_quad,'testNonLinear.in');
impact.writeInputFile(beam,lattice_quad);
impact.run()
pData_nonlinear1 = impact.readParticleData(1002,ke,mass,freq)

In [10]:
print(pData_linear1[:,:6]-pData_nonlinear1[:,:6])

[[-8.862e-05 -3.471e-03  7.289e-05  2.929e-03 -3.226e-04  0.000e+00]
 [ 9.017e-05  3.529e-03 -3.565e-05 -1.434e-03 -3.423e-04  0.000e+00]
 [-4.149e-05 -1.651e-03  4.322e-05  1.749e-03 -2.968e-05  0.000e+00]
 [ 5.354e-05  2.123e-03 -5.157e-05 -2.086e-03 -5.164e-05  0.000e+00]
 [-1.304e-04 -5.140e-03  2.740e-05  1.129e-03 -3.687e-04  0.000e+00]
 [-4.785e-05 -1.849e-03 -7.808e-06 -3.067e-04 -2.483e-04  0.000e+00]
 [-1.118e-04 -4.380e-03  2.823e-05  1.142e-03 -4.688e-04  0.000e+00]
 [ 1.505e-04  5.914e-03  1.007e-05  4.011e-04 -6.194e-04  0.000e+00]]


In [11]:
%%bash
diff testLinear.in testNonLinear.in

3c3
< 6 8 1 0 1 
---
> 6 8 2 0 1 


##### check with liner matrix theory

In [30]:
np.set_printoptions(precision=6)
quad = lattice_quad[2]
K = impact.util.B1toK(quad.B1,ke,mass)
Mquad = impact.util.Mquad(quad.length,quad.B1)
pData = pData_drift0[0,:2]
pData1 = np.matmul(Mquad,pData)
print(pData1)
print(pData_linear1[0,:2])
print(pData_nonlinear1[0,:2])

[[-0.002016  0.003529]]
[-0.002016  0.003529]
[-0.001927  0.007   ]
